# 영어 > 프랑스어 번역기 (단어 레벨)

In [22]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import re    
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

In [23]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수:', len(lines))
lines.sample(5)

전체 샘플의 수: 178009


,eng,fra,cc
16465,That's so ironic.,C'est tellement ironique !,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
99069,I'm not the one who should go.,Je ne suis pas celui qui devrait y aller.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
19127,I don't like eggs.,Je n'aime pas les œufs.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
59781,You should have seen me.,Tu aurais dû me voir !,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
136745,The man caught the girl by the wrist.,L'homme attrapa la fille par le poignet.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [24]:
lines = lines[['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
3807,I thought so.,Je m'en suis douté.
2291,I like this.,Je l'apprécie.
31886,The idea is not bad.,L'idée n'est pas mauvaise.
1538,It's clean.,C'est propre.
6888,We want to go.,Nous voulons y aller.


In [25]:
lines_np_eng= lines['eng'].to_numpy()
lines_np_fra= lines['fra'].to_numpy()
lines_np_eng

array(['Go.', 'Hi.', 'Hi.', ..., 'What would Tom need?',
       'What would you like?', 'What would you like?'], dtype=object)

## 전처리

In [26]:
sos_token = '<start> '
eos_token = ' <end>'

def preprocess_line(line, plus_token = True):
    # 소문자로 변경하기
    line = line.lower().strip()
    # 구두점(Punctuation)을 단어와 분리하기
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line)

    line = line.strip()
    
    if plus_token == True:
        line = sos_token + line + eos_token
    
    return line

In [27]:
def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=7000,  
        filters=' ',   
        oov_token="<unk>"  
    )
    tokenizer.fit_on_texts(corpus)  

    tensor = tokenizer.texts_to_sequences(corpus)   

    return tensor, tokenizer

In [28]:
eng_lines = []
fra_lines = []

# eng_lines.append(lines.eng.apply(lambda x : preprocess_line(x,plus_token = False)))
# fra_lines.append(lines.fra.apply(lambda x : preprocess_line(x),))

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    eng_lines.append(preprocess_line(eng, plus_token = False))
    fra_lines.append(preprocess_line(fra))

In [29]:
np.shape(eng_lines)

(33000,)

In [30]:
eng_tensor, eng_tokenizer = tokenize(eng_lines)
fra_tensor, fra_tokenizer = tokenize(fra_lines)
fra_tensor[:10]

[[2, 93, 13, 3],
 [2, 1070, 13, 3],
 [2, 1070, 4, 3],
 [2, 930, 13, 3],
 [2, 2192, 13, 3],
 [2, 53, 7, 3],
 [2, 11, 621, 13, 3],
 [2, 101, 510, 13, 3],
 [2, 19, 176, 13, 3],
 [2, 2654, 4, 3]]

In [31]:
encoder_input = eng_tensor
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index['<end>']] for line in fra_tensor]
# 시작 토큰 제거
decoder_target =[[char for char in line if char != fra_tokenizer.word_index['<start>']] for line in fra_tensor]

In [32]:
def pad_tensor(tensor):
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = max(num_tokens)
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    tensor = pad_sequences(tensor, padding='post', maxlen=maxlen)  
    return tensor

In [33]:
encoder_input = pad_tensor(encoder_input)
decoder_input = pad_tensor(decoder_input)
decoder_target = pad_tensor(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 16)
프랑스어 출력데이터의 크기(shape) : (33000, 16)


In [34]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1

max_eng_seq_len = encoder_input.shape[1]
max_fra_seq_len = decoder_input.shape[1]

In [35]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [36]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 16)
(30000, 16)
(3000, 8)
(3000, 16)
(3000, 16)


## 모델 만들기

In [37]:
embedding_size = 50
hidden_size = 50
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [38]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [39]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)


In [40]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [41]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     233200      encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     366400      decoder_input[0][0]              
____________________________________________________________________________________________

In [42]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=15)

Epoch 1/15
938/938 [==============================] - 12s 13ms/step - loss: 2.1249 - val_loss: 1.6365
Epoch 2/15
938/938 [==============================] - 11s 12ms/step - loss: 1.5692 - val_loss: 1.4816
Epoch 3/15
938/938 [==============================] - 11s 11ms/step - loss: 1.4500 - val_loss: 1.3930
Epoch 4/15
938/938 [==============================] - 11s 11ms/step - loss: 1.3802 - val_loss: 1.3391
Epoch 5/15
938/938 [==============================] - 11s 11ms/step - loss: 1.3317 - val_loss: 1.2996
Epoch 6/15
938/938 [==============================] - 11s 11ms/step - loss: 1.2947 - val_loss: 1.2659
Epoch 7/15
938/938 [==============================] - 11s 11ms/step - loss: 1.2649 - val_loss: 1.2384
Epoch 8/15
938/938 [==============================] - 11s 11ms/step - loss: 1.2404 - val_loss: 1.2196
Epoch 9/15
938/938 [==============================] - 11s 11ms/step - loss: 1.2173 - val_loss: 1.1948
Epoch 10/15
938/938 [==============================] - 11s 11ms/step - loss: 1.199

## 모델 테스트

In [59]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 50)          233200    
_________________________________________________________________
masking_2 (Masking)          (None, None, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 253,400
Trainable params: 253,400
Non-trainable params: 0
_________________________________________________________________


In [60]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [61]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [62]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 50)     366400      decoder_input[0][0]              
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 50)]         0                                            
____________________________________________________________________________________________

In [63]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [64]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [65]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [67]:
for seq_index in [1,3,713,2114,735]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: let s take a look . 
정답 문장: jetons y un coup d il . 
번역기가 번역한 문장:  un un un . . . 
-----------------------------------
입력 문장: i see you are awake . 
정답 문장: je vois que tu es r veill . 
번역기가 번역한 문장:  je vous que tu 
-----------------------------------
입력 문장: count up to thirty . 
정답 문장: compte jusqu trente . 
번역기가 번역한 문장:  l les . . . . . 
-----------------------------------
입력 문장: we broke up . 
정답 문장: nous nous sommes s par s . 
번역기가 번역한 문장:  nous nous . . . 
-----------------------------------
입력 문장: yours was better . 
정답 문장: la tienne tait meilleure . 
번역기가 번역한 문장:  la tait tait la 


## 느낀점
1. 코드를 복붙 위주로 많이 하다보니까, 틀렸을 때 뭐가 틀렸는지 잘 모르겠다.
2. 아직 개념이 정확하지 않은 것 같다.
3. 다시 복습이 필요하다...